In [72]:
import tensorflow

In [73]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,LSTM,Dense
import numpy as np

In [74]:
batch_size=64    #batch size for training
epochs=10        #No. of epochs
latent_dim=256     #Dimensions of encoding
num_samples=10000  #No. of train samples

In [75]:
data_path="/content/fra.txt"

In [76]:
#vectorize the path
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()

In [77]:
with open(data_path,"r",encoding='utf-8') as f:
  lines=f.read().split("\n")
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text,_=line.split("\t")
  target_text="\t"+target_text+"\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


In [78]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\tJe comprends.\n',
 '\tAha.\n',
 "\tJ'essaye.\n",
 "\tJ'ai gagné !\n",
 "\tJe l'ai emporté !\n",
 '\tJ’ai gagné.\n',
 '\tOh non !\n',
 '\tCalme-toi.\n',
 '\tDétends-to

In [79]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Fire!',
 'Help!',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Attack!',
 'Attack!',
 'Attack!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Get up.',
 'Get up.',
 'Go now.',
 'Go now.',
 'Go now.',
 'Got it!',
 'Got it!',
 'Got it!',
 'Got it?',
 'Got it?',
 'Got it?',
 'Hop in.',
 'Hop in.',
 'Hug me.',
 'Hug me.',
 'I fell.',
 'I fell.',
 'I fled.',
 'I knit.',
 'I

In [80]:
input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_seq_length=max([len(txt) for txt in target_texts ])

In [81]:
print("Number of samples:",len(input_texts))
print("Numer of Unique input tokens:",num_encoder_tokens)
print("Numer of unique output tokens:",num_decoder_tokens)
print("Max sequences length for input:",max_encoder_seq_length)
print("Max sequences length for output:",max_decoder_seq_length)

Number of samples: 10000
Numer of Unique input tokens: 71
Numer of unique output tokens: 93
Max sequences length for input: 15
Max sequences length for output: 59


In [82]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([(char,i) for i,char in enumerate(target_characters)])

In [83]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69,
 'é': 70}

In [84]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype="float32")
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_length,num_decoder_tokens),dtype="float32")

In [85]:
#one_hot encoding

for i ,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[' ']]=1
  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]]=1
    if t>0:
      decoder_target_data[i,t-1,target_token_index[char]]=1
  decoder_input_data[i,t+1:,target_token_index[' ']]=1
  decoder_target_data[i,t:,target_token_index[" "]]=1  

In [86]:
encoder_input_data[0].shape

(15, 71)

In [87]:
#define an input sequence and setup encoder

encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True)
encoder_outputs,state_h,state_c=encoder(encoder_inputs)

#we discards encoder_outputs and only keep state as encoders have no output

encoder_states=[state_h,state_c]

In [88]:
#setup decoder

decoder_inputs=Input(shape=(None,num_decoder_tokens))
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense=Dense(num_decoder_tokens,activation="softmax")
decoder_outputs=decoder_dense(decoder_outputs)

In [89]:
#Define the model that will turn encoder_input and decoder_input into decoder_target_data

model=Model([encoder_inputs,decoder_inputs],decoder_outputs)
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=['accuracy'])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/10
125/125 [==============================] - 40s 301ms/step - loss: 1.4840 - accuracy: 0.7048 - val_loss: 1.0272 - val_accuracy: 0.7134
Epoch 2/10
125/125 [==============================] - 37s 296ms/step - loss: 0.8675 - accuracy: 0.7659 - val_loss: 0.8293 - val_accuracy: 0.7681
Epoch 3/10
125/125 [==============================] - 36s 292ms/step - loss: 0.6850 - accuracy: 0.8087 - val_loss: 0.6996 - val_accuracy: 0.7979
Epoch 4/10
125/125 [==============================] - 37s 293ms/step - loss: 0.5864 - accuracy: 0.8302 - val_loss: 0.6346 - val_accuracy: 0.8138
Epoch 5/10
125/125 [==============================] - 37s 294ms/step - loss: 0.5355 - accuracy: 0.8426 - val_loss: 0.5906 - val_accuracy: 0.8290
Epoch 6/10
125/125 [==============================] - 37s 293ms/step - loss: 0.4969 - accuracy: 0.8541 - val_loss: 0.5604 - val_accuracy: 0.8360
Epoch 7/10
125/125 [==============================] - 37s 294ms/step - loss: 0.4654 - accuracy: 0.8629 - val_loss: 0.5441 - val_ac

In [93]:
#defining sample model
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [96]:
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items() )

In [97]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

    

In [98]:
for seq_index in range(100):
      input_seq=encoder_input_data[seq_index:seq_index+1]
      decoded_sentence=decode_sequence(input_seq)
      print("--")
      print("Input Sentences:",input_texts[seq_index])
      print("Decoded Sentences:",decoded_sentence)

--
Input Sentences: Go.
Decoded Sentences: Dégage !

--
Input Sentences: Go.
Decoded Sentences: Dégage !

--
Input Sentences: Go.
Decoded Sentences: Dégage !

--
Input Sentences: Hi.
Decoded Sentences: Tu la chait !

--
Input Sentences: Hi.
Decoded Sentences: Tu la chait !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run!
Decoded Sentences: Fite !

--
Input Sentences: Run.
Decoded Sentences: Fite !

--
Input Sentences: Run.
Decoded Sentences: Fite !

--
Input Sentences: Run.
Decoded Sentences: Fite !

--
Input Sentences: Run.
Decoded Sentences: Fite !

--
Input Sentences: Run.
Decoded Sentences: Fite !

--
Input Sentences: Run.
Decoded Sentences: Fite 